In [1]:
import os
import sys
import matplotlib.pyplot as plt
# add local lib to sys path for relative import
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# import pandas as pd
from nvis import vis
from nvis.log.common import read_pickle, TensorType
import re
import pandas as pd
df = read_pickle('../test-data/numerics_df_v3.pkl')



vis.interactive(
    f= vis.scalar_line,
    width=1000,
    df=df,
    tt=TensorType.Activation,
    layer='output',
    col_wrap = 3,
    scalar_metric=['rm2','min_abs']
)



Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1000px'), outputs=({'output_type': 'display_data', 'data…